In [1]:
# Import dependancies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import csv
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
import matplotlib.pyplot as plt

### Get Vaccine Data from CDC

In [2]:
# Get Vaccinations Table from CDC website
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit("https://covid.cdc.gov/covid-data-tracker/#vaccinations")
vaccine_html = browser.html
browser.click_link_by_id("vaccinations-table-toggle")
cdc_html = browser.html
cdc_parsed = bs(cdc_html, "html.parser")
table = cdc_parsed.select("table")
browser.quit()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\coled\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [3]:
# Read the CDC Vaccine HTML Table
vaccines_df = pd.read_html(str(table))[0]

In [4]:
# Change "New York State" to "New York" for data merging
vaccines_df.loc[vaccines_df["State/Territory/Federal Entity"] == "New York State", "State/Territory/Federal Entity"] = "New York"

In [5]:
# Rename "State/Territory/Federal Entity" column to "state" for data merging
vaccines_df = vaccines_df.rename(columns={"State/Territory/Federal Entity": "state"})

In [6]:
# Overwrite Latest Vaccine CSV
vaccines_df.to_csv("../data/cdc-vaccine-data.csv")

### NYT / Rearc COVID Data

In [7]:
# Read Daily COVID-19 CSV from AWS S3 Bucket - Rearc / NY Times Data 
us_covid_cases_data = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-states/us-states.csv", stream=True)

In [8]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/us-covid-data.csv",  "wb") as file:
    file.write(us_covid_cases_data.content)

In [9]:
# Read US COVID Data CSV for data merging and cleaning 
us_covid_cases_df = pd.read_csv("../data/us-covid-data.csv")

In [10]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20084,2021-03-02,Virginia,51,578559,8943
20085,2021-03-02,Washington,53,344516,5056
20086,2021-03-02,West Virginia,54,132184,2301
20087,2021-03-02,Wisconsin,55,618310,7055


# State Overview Data

In [11]:
# Get yesterday's date, which is the latest data 
today = dt.date.today()
yesterday = today - dt.timedelta(days = 1)
latest_data = yesterday.strftime("%Y-%m-%d")

In [12]:
# Get latest total cases by state for latest date
us_total_cases_to_date_df = us_covid_cases_df.groupby(["date", "state"]).sum()
us_total_cases_to_date_df.reset_index(inplace=True)
us_total_cases_to_date_df = us_total_cases_to_date_df.loc[us_total_cases_to_date_df["date"] == latest_data]

In [13]:
# Read State Lat-Lon CSV and rename columns for easier cleaning and merging
state_latlons_df = pd.read_csv("../data/statelatlong.csv")
state_latlons_df.rename(columns={"State": "abbr", "Latitude": "lat", "Longitude": "lon", "City": "state"}, inplace=True)

In [14]:
# Read State Population CSV for cleaning and merging
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")

In [15]:
# Merge, rename, and focus on defined columns
state_overview_master_df = pd.merge(state_pop_df, state_latlons_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["state", "abbr", "lat", "lon", "population"]]

In [16]:
# Merge with us_total_cases_to_date_df and focus on defined columns
state_overview_master_df = pd.merge(state_overview_master_df, us_total_cases_to_date_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths"]]

In [17]:
# Merge with Vaccination data, rename columns, focus data columns
state_overview_master_df = pd.merge(state_overview_master_df, vaccines_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df.rename(columns={"Total Doses Administered by State where Administered": "total_doses_administered", "Doses Administered per 100k by State where Administered": "doses_administered_per_100k"})
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths", "total_doses_administered", "doses_administered_per_100k"]]

In [18]:
# Add Empty Column for New Calculation
state_overview_master_df["percent_vaccinated"] = ""

In [19]:
# Calcuation for percent vaccinated by state
for index, row in state_overview_master_df.iterrows():
    population = row["population"]
    vaccinated = row["total_doses_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_overview_master_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [20]:
# Convert percent vaccinated into a float
state_overview_master_df = state_overview_master_df.astype({"percent_vaccinated": 'float64'})

In [21]:
state_overview_master_df.to_csv("../data/state-master-data.csv")

In [90]:
state_overview_master_df

,date,state,abbr,lat,lon,population,fips,cases,deaths,total_doses_administered,doses_administered_per_100k,percent_vaccinated
0,2021-03-02,Alabama,AL,32.601011,-86.680736,4903185,1,494421,9991,972161,19827.0,19.83
1,2021-03-02,Alaska,AK,61.302501,-158.775020,731545,2,58383,290,269941,36900.0,36.90
2,2021-03-02,Arizona,AZ,34.168219,-111.930907,7278717,4,819281,16065,1889742,25963.0,25.96
3,2021-03-02,Arkansas,AR,34.751928,-92.131378,3017804,5,322949,5254,670815,22229.0,22.23
4,2021-03-02,California,CA,37.271875,-119.270415,39512223,6,3577497,52774,9497125,24036.0,24.04
5,2021-03-02,Colorado,CO,38.997934,-105.550567,5758736,8,432939,6054,1428854,24812.0,24.81
6,2021-03-02,Connecticut,CT,41.518784,-72.757507,3565287,9,283128,7658,1040154,29174.0,29.17
7,2021-03-02,Delaware,DE,39.145251,-75.418921,973764,10,87193,1426,225417,23149.0,23.15
8,2021-03-02,District of Columbia,DC,38.899349,-77.014567,705749,11,40767,1019,199212,28227.0,28.23
9,2021-03-02,Florida,FL,27.975728,-83.833017,21477737,12,1918092,31134,5077891,23643.0,23.64


# County Overview Data

In [22]:
populations_county = pd.read_csv("../data/census-bureau-population-by-county.csv")
poverty_county = pd.read_csv("../data/poverty-and-median-household-income-data-by-us-county-2019.csv")
state_keys = pd.read_csv("../data/state-names-codes.csv")

In [23]:
poverty_county = poverty_county[["Postal Code", "county", "Poverty Estimate, All Ages", "Median Household Income"]]
poverty_county = poverty_county.rename(columns={"Postal Code": "code", "Poverty Estimate, All Ages": "poverty_est_all_ages", 
                                              "Median Household Income": "median_household_income"})
poverty_county["county"] = poverty_county["county"].str.rstrip()

In [24]:
state_keys = state_keys.rename(columns={"Code": "code", "State": "state"})
state_keys = state_keys[["state", "code"]]

In [25]:
poverty_county = pd.merge(poverty_county, state_keys, how="inner", on="code")
poverty_county.drop(columns={"state"}, inplace=True)

In [26]:
populations_county = populations_county.merge(state_keys, how="inner", on="state")
populations_county = populations_county[["code", "county", "state", "2019_population_est"]]
populations_county["code"] = populations_county["code"].str.rstrip()
populations_county["county"] = populations_county["county"].str.rstrip()

In [27]:
merged_data = populations_county.merge(poverty_county, how='inner', left_on=["code", "county"], right_on=["code", "county"])

In [28]:
merged_data.dropna(how="any", inplace=True)

### Retrieve County Cases Data

In [29]:
county_cases = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-counties/us-counties.csv", stream=True)

In [30]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/county-covid-data.csv",  "wb") as file:
    file.write(county_cases.content)

In [31]:
# Read US COVID Data CSV for data merging and cleaning 
us_county_cases = pd.read_csv("../data/county-covid-data.csv")

In [32]:
us_county_cases = us_county_cases.groupby(["date", "state", "county"]).sum()
us_county_cases.reset_index(inplace=True)

In [33]:
latest_covid_cases = us_county_cases.loc[us_county_cases["date"] == latest_data]

In [34]:
county_cases = latest_covid_cases.merge(merged_data, how="left")

In [35]:
county_cases.dropna(how="any", inplace=True)

In [36]:
county_cases.to_csv("../data/county-master.csv")

In [58]:
latest_county_cases = county_cases.loc[county_cases["date"] == latest_data]

In [59]:
latest_county_cases

,date,state,county,fips,cases,deaths,code,2019_population_est,poverty_est_all_ages,median_household_income
0,2021-03-02,Alabama,Autauga,1001.0,6303,91.0,AL,55869.0,"6,723","58,233"
1,2021-03-02,Alabama,Baldwin,1003.0,19790,284.0,AL,223234.0,"22,360","59,871"
2,2021-03-02,Alabama,Barbour,1005.0,2124,51.0,AL,24686.0,"5,909","35,972"
3,2021-03-02,Alabama,Bibb,1007.0,2454,60.0,AL,22394.0,"4,101","47,918"
4,2021-03-02,Alabama,Blount,1009.0,6106,127.0,AL,57826.0,"9,324","52,902"
...,...,...,...,...,...,...,...,...,...,...
3262,2021-03-02,Wyoming,Sweetwater,56037.0,3774,35.0,WY,42343.0,"3,453","80,639"
3263,2021-03-02,Wyoming,Teton,56039.0,3359,9.0,WY,23464.0,"1,396","98,837"
3264,2021-03-02,Wyoming,Uinta,56041.0,2062,12.0,WY,20226.0,"1,699","70,756"
3265,2021-03-02,Wyoming,Washakie,56043.0,884,26.0,WY,7805.0,845,"55,122"


## Daily Cases

In [38]:
# Convert Dates to pd.Datetimes
us_covid_cases_df["date"] = pd.to_datetime(us_covid_cases_df["date"])

In [39]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20084,2021-03-02,Virginia,51,578559,8943
20085,2021-03-02,Washington,53,344516,5056
20086,2021-03-02,West Virginia,54,132184,2301
20087,2021-03-02,Wisconsin,55,618310,7055


In [40]:
# Create Daily New Cases Dataframe
daily_new_cases_df = us_covid_cases_df.groupby("date").sum()

In [41]:
daily_new_cases_df.reset_index(inplace=True)

In [42]:
# Add new Empty Columns for Daily New Cases and Deaths
daily_new_cases_df["daily_new_cases"] = ""
daily_new_cases_df["daily_new_deaths"] = ""

In [43]:
previous_day_cases = 0
previous_day_deaths = 0
    
for index, row in daily_new_cases_df.iterrows():
    
    # Cases
    new_cases = row["cases"] - previous_day_cases
    previous_day_cases = row["cases"]
    daily_new_cases_df.at[index, "daily_new_cases"] = new_cases
    
    # Deaths
    new_deaths = row["deaths"] - previous_day_deaths
    previous_day_deaths = row["deaths"]
    daily_new_cases_df.at[index, "daily_new_deaths"] = new_deaths

In [76]:
daily_new_cases_df["3_month_immunity_est"] = daily_new_cases_df.rolling(100)["daily_new_cases"].sum()

In [78]:
daily_new_cases_df["6_month_immunity_est"] = daily_new_cases_df.rolling(190)["daily_new_cases"].sum()

In [79]:
daily_new_cases_df

,date,fips,cases,deaths,daily_new_cases,daily_new_deaths,3_month_immunity_est,6_month_immunity_est
0,2020-01-21,53,1,0,1,0,NaN,NaN
1,2020-01-22,53,1,0,0,0,NaN,NaN
2,2020-01-23,53,1,0,0,0,NaN,NaN
3,2020-01-24,70,2,0,1,0,NaN,NaN
4,2020-01-25,76,3,0,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...
402,2021-02-26,1762,28509327,510283,78262,2176,16895211.0,22921865.0
403,2021-02-27,1762,28572021,511850,62694,1567,16770390.0,22935530.0
404,2021-02-28,1762,28622946,512979,50925,1129,16622434.0,22941429.0
405,2021-03-01,1762,28679618,514404,56672,1425,16507161.0,22965768.0


In [86]:
daily_new_cases_df.to_csv("../data/state-daily-stats.csv")

### Set Up NYT Totals Master DF

In [69]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20084,2021-03-02,Virginia,51,578559,8943
20085,2021-03-02,Washington,53,344516,5056
20086,2021-03-02,West Virginia,54,132184,2301
20087,2021-03-02,Wisconsin,55,618310,7055


## Amazon RDS Updates

In [48]:
# Config Variables, and SQLalchemy
from config import endpoint, username, password
from sqlalchemy import create_engine

In [49]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [50]:
# 50 States & D.C. Data
state_overview_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [51]:
engine.execute('ALTER TABLE master_table ADD PRIMARY KEY (state);')

In [87]:
# Daily Cases Data
daily_new_cases_df.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)

In [88]:
engine.execute('ALTER TABLE daily_new_cases ADD PRIMARY KEY (date);')

In [54]:
# County Data
county_cases.to_sql("county_cases_master", index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE county_cases_master ADD PRIMARY KEY (index);')

In [55]:
# Raw NTY Data
us_covid_cases_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE nyt_table ADD PRIMARY KEY (index);')

In [89]:
# latest_county_cases.to_sql('county_cases_by_day', index=True, if_exists='append', con=connection)